# Unpacking Original VotePageInfo.json File

## This python notebook packages a series of steps into one function, which results in a new pickled Dataframe object file being generated with the unpacked json data

## The final helper function has 3 internal steps:

### 1) Import the JSON file into a Pandas notebook
### 2) Unpack the nested JSON objects into indivdual dataframe columns, and delete the old, nested columns
### 3) Save the new pickled file (which is easier to import and work with)

In [1]:
import pandas as pd
import json
import pickle

## 1) Import the Routes Json File

In [2]:
def import_json_file(filepath):
    # Import file into pandas dataframe
    with open(filepath) as file:
        reader = file.read()
        votesDict = dict(json.loads(reader))
        votes = pd.DataFrame.from_dict(votesDict, orient='index')
    return votes

### Functions to extract Data

In [3]:
# Extract the user star votes and return a list of lists with format"
# [ ['userUrl_1_climb1', 'User_1_Vote_climb1'], ['userUrl_2_climb1', 'User_2_Vote_climb1'], ...]
def get_user_star_votes(votes_dataframe):
    votes_dataframe['userStarVotes'] = [votes_dataframe['starRatings'][item]['ratings'] for item in range(len(votes_dataframe['starRatings']))]
    userStarVotes = []
    it = iter(votes_dataframe)
    for i in it:
        print(i)
    for vote in votes_dataframe.userStarVotes:
        climbVotes = []
        try:
            for item in vote:
                climbVotes.append([item, vote[item]['rating']])
            userStarVotes.append(climbVotes)
        except:
            userStarVotes.append(None)
    votes_dataframe.drop(['userStarVotes'], axis=1)

    return userStarVotes

# Extract the difficulty ratings and return a list of lists
def get_difficulty_ratings(votes_dataframe):
    votes_dataframe['diffRatings'] = [votes_dataframe['difficultyRatings'][item]['ratings'] for item in range(len(votes_dataframe['difficultyRatings']))]
    difficultyRatings = []
    for vote in votes_dataframe.diffRatings:
        climbVotes = []
        try:
            for item in vote:
                climbVotes.append([item, vote[item]['rating']])
            difficultyRatings.append(climbVotes)
        except:
            difficultyRatings.append(climbVotes)
    
    return difficultyRatings

# Extract a route's ticks as a list of dictionaries. Each list member is one dictionary
def get_ticks(votes_dataframe):
    votes_dataframe['ticks'] = [votes_dataframe['ticks'][item] for item in range(len(votes_dataframe['ticks']))]
    ticks = []
    for route in votes_dataframe['ticks']:
        ticks.append(route)
    return ticks
            
# Extract a route's ticks as a list of dictionaries. Each list member is one dictionary
def get_toDo_list(votes_dataframe):
    votes_dataframe['toDoList'] = [votes_dataframe['toDoList'][item] for item in range(len(votes_dataframe['toDoList']))]
    toDoList = []
    for route in votes_dataframe['toDoList']:
        toDoList.append(route['users'])
    return toDoList


## 2) Unpack nested JSON objects

In [4]:
# Get the data
def unpack_data(votes):
    votes['starRatings'] = get_user_star_votes(votes)
    votes['difficultyRatings'] = get_difficulty_ratings(votes)
    votes['ticks'] = get_ticks(votes)
    votes['toDoList'] = get_toDo_list(votes)
    votes.drop(columns=['userStarVotes', 'diffRatings'], axis=1, inplace=True)
    return votes

## 3) Save new file with unpacked JSON objects as individual columns

In [5]:
# Save the newly unpacked votes file
def save_vote_dataframe(votes_dataframe, destPath):
    votes_dataframe.to_pickle(destPath)

## 4) Package into a helper function to do 1, 2 and 3 in one step

In [6]:
def unpack_votes(filepath, destPath):
    votes = import_json_file(filepath)
    votes = unpack_data(votes)
    save_vote_dataframe(votes, destPath)

filepath = './Data/votePageData.json'
destPath = './Data/pickledVotes_unpacked'
unpack_votes(filepath, destPath)

name
starRatings
difficultyRatings
toDoList
ticks
userStarVotes
